# Monte Carlo Simulation
## Pascal's Problem

Write a program to simulate the probability of rolling a double 6 in 24 consective rolls.

*   On the first roll the probability of rolling a 6 on each die is 1/6, so the probability of rolling a 6 with both dice is 1/36. 
*   Therefore, the probability of not rolling a double 6 on the first roll is 1 - 1/36 = 35/36. 
*   Therefore the probability of not rolling a double 6 twenty-four consecutive times is (35/36)24, nearly 0.51, and therefore the probability of rolling a double 6 is 1 - (35/36)24, about 0.49. In the long run it would not be profitable to bet on rolling a double 6 within twenty-four rolls.


In [36]:
import random

def rollDie():
    return random.choice([1,2,3,4,5,6])

def checkPascal(numTrials):
    """Assumes numTrials an in > 0
    Prints an estimate of the probability of winning"""
    numWins = 0
    for i in range(numTrials):
        for j in range(24):
            d1 = rollDie()
            d2 = rollDie()
            if d1 == 6 and d2 == 6:
                numWins += 1
                break
    print('Probability of winning =', numWins / numTrials)

In [37]:
# Estimate with Simulation
checkPascal(1000000)

Probability of winning = 0.490652


In [38]:
# Check Work Arithmatically 
1 - (35.0/36.0)**24

0.4914038761309034

## Craps - Pass / Don't Pass

In [13]:
class CrapsGame(object):
    def __init__(self) -> None:
        self.passWins, self.passLosses = 0, 0
        self.dpWins, self.dpLosses, self.dpPushes = 0, 0, 0
    
    # def playHand(self):
    #     # Straight forward approach to playHand, less efficient.
    #     throw = rollDie() + rollDie()
    #     if throw == 7 or throw == 11:
    #         self.passWins += 1
    #         self.dpLosses += 1
    #     elif throw == 2 or throw == 3 or throw == 12:
    #         self.passLosses += 1
    #         if throw == 12:
    #             self.dpPushes += 1
    #         else:
    #             self.dpWins += 1
    #     else:
    #         point = throw
    #         while True:
    #             throw = rollDie() + rollDie()
    #             if throw == point:
    #                 self.passWins += 1
    #                 self.dpLosses += 1
    #                 break
    #             elif throw == 7:
    #                 self.passLosses += 1
    #                 self.dpWins += 1
    #                 break
    
    def playHand(self):
        #An alternative, faster, implementation of playHand
        pointsDict = {4:1/3, 5:2/5, 6:5/11, 8:5/11, 9:2/5, 10:1/3}
        throw = rollDie() + rollDie()
        if throw == 7 or throw == 11:
            self.passWins += 1
            self.dpLosses += 1
        elif throw == 2 or throw == 3 or throw == 12:
            self.passLosses += 1
            if throw == 12:
                self.dpPushes += 1
            else:
                self.dpWins += 1
        else:
            if random.random() <= pointsDict[throw]: # point before 7
                self.passWins += 1
                self.dpLosses += 1
            else: # 7 before point
                self.passLosses += 1
                self.dpWins += 1

                            
    def passResults(self):
        return (self.passWins, self.passLosses)
    
    def dpResults(self):
        return (self.dpWins, self.dpLosses, self.dpPushes)

In [14]:
def variance(X):
    """Assumes that X is a list of numbers.
    Returns the standard deviation of X"""
    mean = sum(X)/len(X)
    tot = 0.0
    for x in X:
        tot  += (x-mean)**2
    return tot/len(X)

def stdDev(X):
    """Assumes that X is a list of numbers.
    Returns the standard deviation of X"""
    return variance(X)**0.5

In [15]:
def crapsSim(handsPerGame, numGames):
    """Assumes handsPerGame and numGames are ints > 0
    Play numGames games of handsPerGame hands; print results"""
    games = []
    
    # Play numGames games
    for t in range(numGames):
        c = CrapsGame()
        for i in range(handsPerGame):
            c.playHand()
        games.append(c)
    
    # Produce statistics for each game
    pROIPerGame, dpROIPerGame = [], []
    for g in games:
        wins, losses = g.passResults()
        pROIPerGame.append((wins - losses)/float(handsPerGame))
        wins, losses, pushes = g.dpResults()
        dpROIPerGame.append((wins - losses)/float(handsPerGame))
        
    # Produce and print summary statistics
    # Pass Line
    meanROI = str(round((100*sum(pROIPerGame)/numGames), 4)) + '%'
    sigma  = str(round(100*stdDev(pROIPerGame), 4)) + '%'
    print('Pass:', 'Mean ROI =', meanROI, 'Std. Dev. =', sigma)
    # Produce and print summary statistics
    # Don't Pass Line
    meanROI = str(round((100*sum(dpROIPerGame)/numGames), 4)) + '%'
    sigma  = str(round(100*stdDev(dpROIPerGame), 4)) + '%'
    print('Pass:', 'Mean ROI =', meanROI, 'Std. Dev. =', sigma)

In [35]:
crapsSim(20, 10)

Pass: Mean ROI = -5.0% Std. Dev. = 21.095%
Pass: Mean ROI = 3.5% Std. Dev. = 21.5697%


In [20]:
crapsSim(1000000, 10)

Pass: Mean ROI = -1.3688% Std. Dev. = 0.0696%
Pass: Mean ROI = -1.4113% Std. Dev. = 0.0747%


In [18]:
crapsSim(20, 1000000)

Pass: Mean ROI = -1.4353% Std. Dev. = 22.3562%
Pass: Mean ROI = -1.3441% Std. Dev. = 22.0452%


## Using Table Lookup to Improve Performance
I modified the class above to achieve faster performace over the looping.

## Weighted Die - Bias
A die that favored 5 over 2.

In [39]:
def rollDie():
    return random.choice([1,1,2,3,3,4,4,5,5,5,6,6])

In [42]:
crapsSim(1000000, 10)

Pass: Mean ROI = 2.2973% Std. Dev. = 0.0518%
Pass: Mean ROI = -5.0739% Std. Dev. = 0.0491%
